In [1]:
import numpy as np
from susy_qm import calculate_wz_hamiltonian
import pennylane as qml
from pennylane import numpy as pnp
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import json
import os

In [15]:
# Parameters
N = 3
cutoff = 2
a = 1.0
potential = "quadratic"
boundary_condition = 'periodic'
c = -0.2

H = calculate_wz_hamiltonian(cutoff, N, a, potential, boundary_condition, c)

In [16]:
eigenvalues, eigenvectors = np.linalg.eig(H)

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

min_eigenvector = np.asarray(min_eigenvector)

In [17]:
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [5]:
sel_num_layers = 2  # Number of entangling layers
sel_params_shape = qml.StronglyEntanglingLayers.shape(n_layers=sel_num_layers, n_wires=num_qubits)

ran_num_layers = 3  # Number of entangling layers
ran_params_shape = qml.RandomLayers.shape(n_layers=ran_num_layers, n_rotations=num_qubits)


dev = qml.device('default.qubit', wires=num_qubits, shots=None)
@qml.qnode(dev)
def ansatz(sel_params, rand_params, seed):
    qml.StronglyEntanglingLayers(sel_params, wires=range(num_qubits), imprimitive=qml.CZ)
    qml.RandomLayers(rand_params, wires=range(num_qubits), imprimitive=qml.CZ, seed=seed)
    return qml.state()

In [18]:
dev = qml.device('default.qubit', wires=num_qubits, shots=None)
@qml.qnode(dev)
def ansatz(params):
    param_index=0
    for i in range(0, num_qubits):
        qml.RY(params[param_index], wires=i)
        param_index += 1

    # Apply entanglement
    for j in reversed(range(1, num_qubits)):
        qml.CZ(wires=[j - 1, j])
        #qml.CNOT(wires=[j - 1, j])

    # Apply RY rotations
    for k in range(0, num_qubits):
        qml.RY(params[param_index], wires=k)
        param_index += 1
    
    return qml.state()

In [19]:
def cost_function(params, seed):

    sel_params = params[:np.prod(sel_params_shape)]
    ran_params = params[np.prod(sel_params_shape):]
    sel_params = pnp.tensor(sel_params.reshape(sel_params_shape), requires_grad=True)
    ran_params = pnp.tensor(ran_params.reshape(ran_params_shape), requires_grad=True)

    ansatz_state = ansatz(sel_params, ran_params, seed)
    ansatz_state = np.array(ansatz_state)

    overlap = np.vdot(min_eigenvector, ansatz_state)
    overlap_squared = np.abs(overlap)**2
    return 1 - overlap_squared


In [20]:
def overlap_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = ansatz(params)
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  
    
    return (1 - cost)

In [21]:
x0 = np.random.uniform(0, 2 * np.pi, size=2*num_qubits)

overlap_res = minimize(
    overlap_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':10000, 'tol': 1e-8}
)

Running for overlap


In [22]:
overlap_res.fun

np.float64(0.004303764371020202)

In [ ]:
num_tests=1000
maxiter = 10000
costfn = []
paramvalues = []
success = []
iters = []
seeds = []

for i in range(num_tests):

    print(f"Run {i}")

    sel_x0 = np.random.random(sel_params_shape).flatten()
    ran_x0 = np.random.random(ran_params_shape).flatten()

    x0 = np.concatenate((sel_x0, ran_x0))

    seed = np.random.randint(0,1e6)

    res = minimize(
        cost_function,
        x0,
        args=(seed,),
        method= "COBYLA",
        options= {'maxiter':10000}
    )

    costfn.append(res.fun)
    paramvalues.append(res.x)
    success.append(res.success)
    seeds.append(seed)
    iters.append(res.nfev)

Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Run 10
Run 11
Run 12
Run 13
Run 14
Run 15
Run 16
Run 17
Run 18
Run 19
Run 20
Run 21
Run 22
Run 23
Run 24
Run 25
Run 26
Run 27
Run 28
Run 29
Run 30
Run 31
Run 32
Run 33
Run 34
Run 35
Run 36
Run 37
Run 38
Run 39
Run 40
Run 41


KeyboardInterrupt: 

In [8]:
data = {
        "potential": potential,
        "boundary_condition": boundary_condition,
        "cutoff": cutoff,
        "N": N,
        "a": a,
        "c": None if potential == "linear" else c,
        "sel_num_layers": sel_num_layers,
        "ran_num_layers": ran_num_layers,
        "num_tests": num_tests,
        "Optimizer": {
            "name": "COBYLA",
            "maxiter": maxiter
        },
        "results": costfn,
        "params": [x.tolist() for x in paramvalues],
        "num_iters": iters,
        "success": np.array(success, dtype=bool).tolist(),
        "seeds": seeds
    }

In [16]:
[x.real.tolist() for x in min_eigenvector]

[[0.0],
 [4.4209081325530976e-18],
 [-7.276359676923743e-17],
 [-8.559155697704452e-17],
 [-4.898456428862115e-17],
 [2.868202198279665e-17],
 [6.125202667301129e-17],
 [-4.9889079434332134e-17],
 [2.4256007929290534e-16],
 [-0.15140033412944112],
 [0.6736542337628943],
 [1.0030414216471654e-16],
 [-0.151400334129441],
 [-1.69687794896566e-16],
 [1.4992257484519385e-16],
 [2.7351064235659415e-17],
 [5.218258097537777e-17],
 [-0.003983241083314441],
 [0.017723390343239998],
 [3.765081637050305e-18],
 [-0.003983241083314395],
 [2.7468236344804423e-18],
 [-1.0147597207747898e-18],
 [-4.868856229504261e-19],
 [1.5277219400295423e-18],
 [-4.967691394913709e-11],
 [2.2103691720467909e-10],
 [-1.3672016009812426e-18],
 [-4.9676914058864524e-11],
 [1.302029217456873e-19],
 [-4.6572247526772206e-20],
 [-1.0392828476837679e-22],
 [4.20607990856955e-17],
 [-0.15140033412944132],
 [0.6736542337628949],
 [-3.8574245432130877e-17],
 [-0.15140033412944143],
 [-5.848132357790767e-16],
 [-3.71579104972

In [9]:
data

{'potential': 'linear',
 'boundary_condition': 'dirichlet',
 'cutoff': 2,
 'N': 3,
 'a': 1.0,
 'c': None,
 'sel_num_layers': 2,
 'ran_num_layers': 3,
 'num_tests': 1000,
 'Optimizer': {'name': 'COBYLA', 'maxiter': 10000},
 'results': [np.float64(0.09225675753436902),
  np.float64(0.002780705106633863),
  np.float64(0.0028039501027411484),
  np.float64(0.044228805048858955),
  np.float64(0.09237990976471322),
  np.float64(0.09236376736081886),
  np.float64(0.046276521498235135),
  np.float64(0.002803701448185758),
  np.float64(0.002799439223979183),
  np.float64(0.09237540102650266),
  np.float64(0.014835344216599267),
  np.float64(0.0027993436388864534),
  np.float64(0.045854864577792864),
  np.float64(0.04602285604971701),
  np.float64(0.002799322303031593),
  np.float64(0.04565722835346553),
  np.float64(0.0027992699972186896),
  np.float64(0.2963629253855051),
  np.float64(0.046746445924813695),
  np.float64(0.04649447130939732),
  np.float64(0.04425762878888806),
  np.float64(0.092

In [10]:
min(data['results'])

np.float64(0.0001779383310284155)

In [11]:
# Save the variable to a JSON file
path = r"C:\Users\Johnk\Documents\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\Ansatz Testing\Files"
os.makedirs(path, exist_ok=True)

file_path = os.path.join(path, 'data3.json')
with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [3]:
path = r"C:\Users\Johnk\Documents\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\Ansatz Testing\Files"
file_path = os.path.join(path, 'data.json')

with open(file_path, 'r') as json_file:
    d = json.load(json_file)

In [7]:
minindex = np.argmin(d['results'])

In [9]:
d['results'][minindex]

0.0006899799006699903

In [10]:
d['params'][minindex]

[1.3118745084227537,
 -1.5707985641343323,
 1.6175010107166166,
 1.0097622555349521,
 -7.475034880764682e-06,
 2.1269291573193887,
 -0.023766356671799323,
 3.3669420187763245e-05,
 1.3346828549365681,
 2.2846865489872954,
 -0.7853841823167704,
 1.7354997484178898,
 -1.7033735452260925,
 0.6155097592896787,
 -0.006455464910488524,
 -0.23959311623595522,
 1.5707408163865053,
 -0.14965604217777131,
 0.5823425874101905,
 -0.0010043318223806506,
 2.5126052396410437,
 2.589985155747876,
 -0.0026448637502780887,
 1.5920544622690325,
 1.8180187273528017,
 1.5707948970983046,
 0.7648521181888704,
 1.4060705977147667,
 -0.7853615006358943,
 1.203978750607472,
 0.00461939223275388,
 3.1415961786235016,
 -0.3403253593530838,
 1.719223275419387,
 -0.9861012719056566,
 1.5705516753346347,
 -3.0098236808231e-05,
 0.9118395392767223,
 0.45976167198191475,
 2.376725676130097,
 -0.5846970409530099,
 -1.57081243154048,
 1.602576924184963,
 1.570152704716289,
 1.1373248359770398,
 -9.058908587712554e-05,


In [16]:
d['seeds'][minindex]

937297

In [18]:
params = pnp.tensor(d['params'][minindex], requires_grad=True)

In [19]:
cost_function(params, d['seeds'][minindex])

np.float64(0.0006899799006699903)

In [26]:
@qml.qnode(dev)
def cf(params):

    sel_params = params[:np.prod(sel_params_shape)]
    ran_params = params[np.prod(sel_params_shape):]
    sel_params = pnp.tensor(sel_params.reshape(sel_params_shape), requires_grad=True)
    ran_params = pnp.tensor(ran_params.reshape(ran_params_shape), requires_grad=True)

    qml.StronglyEntanglingLayers(sel_params, wires=range(num_qubits), imprimitive=qml.CZ)
    qml.RandomLayers(ran_params, wires=range(num_qubits), seed=d['seeds'][minindex])

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [27]:
cf(params)

tensor(-0.09974079, requires_grad=True)